In [1]:
# Basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Modeling
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.linear_model import LassoCV

# Pipeline
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# Load dataset
train_url = "/kaggle/input/mlbasics-competition-l09-spring2024/train.csv"
df_train = pd.read_csv(train_url, index_col='Date')

test_url = '/kaggle/input/mlbasics-competition-l09-spring2024/test.csv'
df_test = pd.read_csv(test_url, index_col='Date')

# Prepare features and target
X = df_train.drop(columns=['Income'])
y = df_train['Income']

# Convert categorical features to dummy variables
X = pd.get_dummies(X, drop_first=True)
df_test = pd.get_dummies(df_test, drop_first=True)

# Ensure both training and testing data have the same dummy variables
X, df_test = X.align(df_test, join='left', axis=1, fill_value=0)

# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
df_test_scaled = scaler.transform(df_test)

# 使用LassoCV进行特征选择
lasso = LassoCV(cv=5)
lasso.fit(X_scaled, y)

# 获取非零系数的特征
selected_features = np.where(lasso.coef_ != 0)[0]
X_selected = X.iloc[:, selected_features]
df_test_selected = df_test.iloc[:, selected_features]

# Split dataset into training and testing sets
X_train, X_valid, y_train, y_valid = train_test_split(X_selected, y, test_size=0.2)

# Define CatBoost model
catboost_model = CatBoostRegressor(loss_function='RMSE', silent=True)

# Define hyperparameter search space
search_spaces = {
    'iterations': Integer(100, 1000),
    'depth': Integer(1, 10),
    'learning_rate': Real(0.01, 0.3, 'log-uniform'),
    'l2_leaf_reg': Real(1, 10, 'log-uniform'),
    'bagging_temperature': Real(0.0, 1.0),
    'border_count': Integer(32, 255)
}

# Define Repeated K-Fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10)

# BayesSearchCV with cross-validation
opt = BayesSearchCV(
    estimator=catboost_model,
    search_spaces=search_spaces,
    n_iter=150,  # Number of parameter settings that are sampled
    cv=cv,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)

# Perform the search
opt.fit(X_train, y_train)

# Best model from Bayesian optimization
best_catboost = opt.best_estimator_

# Predict on validation set
y_pred_valid = best_catboost.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
print(f'Root Mean Squared Error (RMSE) on validation set: {rmse}')

# Predict on test set
test_predictions = best_catboost.predict(df_test_selected)

# Create submission DataFrame
submission = pd.DataFrame({
    'Date': df_test.index,
    'Income': test_predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)


Root Mean Squared Error (RMSE) on validation set: 3.596904204223408
